# Brain Tumor Segmentation

<span style="font-size: 20px; line-height: 30px;">
<ol>
    <li> Load model </li>
    <li> Load test dataset - Images </li>
    <li> Predict the mask </li>
    <li> Measure the pixels </li>
</ol>
</span>

## Import

In [41]:
import os
import time
import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.utils import CustomObjectScope

## Seeding

In [42]:
os.environ["PYTHONHASHSEED"] = str(42)
np.random.seed(42)
tf.random.set_seed(42)

## Hyperparameters

In [43]:
height = 256
width = 256

## Path

In [44]:
dataset_path = "test-images"
mask_path = os.path.join(dataset_path, "masks")
combine_path = os.path.join(dataset_path, "combine")

model_path = "files/unet.h5"

## Create folder

In [45]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

## Folder to save predicted mask

In [46]:
create_dir(mask_path)
create_dir(combine_path)

## Load model

In [47]:
def dice_loss(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    dice = (2. * intersection + 1e-15) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + 1e-15)
    return 1.0 - dice

In [48]:
with CustomObjectScope({'dice_loss': dice_loss}):
    model = tf.keras.models.load_model(model_path)

In [49]:
model.summary()

Model: "UNET"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 262, 262, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 128, 128, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                               

## Load test dataset: Images

In [50]:
test_x = sorted(glob(os.path.join(dataset_path, "images", "*")))
print("Test Images: ", len(test_x))

Test Images:  306


## Predict the mask & Calculate Pixels

In [54]:
for x in test_x:
    name = x.split("/")[-1]
    
    x = cv2.imread(x, cv2.IMREAD_COLOR)
    img_x = x
    h, w, _ = x.shape
    
    x = cv2.resize(x, (width, height))
    x = x/255.0
    x = np.expand_dims(x, axis=0)
    
    p = model.predict(x)[0]
    p = cv2.resize(p, (w, h))
    p = p > 0.5
    
    foreground_mask = p
    background_mask = 1-p
    
    foreground_mask[foreground_mask == 1]
    background_mask[background_mask == 1]
    
    # print(f"{name:10s} - Foreground Pixels: {foreground_mask.sum():5.0f} - Background Pixels: {background_mask.sum():5.0f}")
    
    cv2.imwrite(os.path.join(mask_path, name), p * 255)
    
    p = np.expand_dims(p, axis=-1) 
    p = np.concatenate([p, p, p], axis=-1) * 255

    p = p.astype(np.float32)
    img_x = img_x.astype(np.float32)
    
    alpha = 0.5
    overlay_image = cv2.addWeighted(p, alpha, img_x, 1-alpha, 0)
    
    line = np.ones((h, 10, 3)) * 255
    
    cat_image = np.concatenate([img_x, line, p, line, overlay_image], axis=1)
    cv2.imwrite(os.path.join(combine_path, name), cat_image)
print("done")

1/1 [==============================] - 0s 42ms/step
done
